> This notebook loads raw data from the MPIL dataset, which are then converted into spectral power densities and compiled as a separate DataFrame and CSV file for each of the two testing conditions (eyes open and eyes closed). The set of subject data is then dowloaded and saved as is.

In [1]:
import numpy as np
import pandas as pd
import mne
from mne.time_frequency import psd_welch
from scipy.integrate import simps
import wget
import os

In [2]:
band_dict = {"delta": [1.5, 4], "theta": [4, 7], "alpha": [7, 12], "beta": [12, 30], "gamma": [30, 45]}
channels = ['Fp1', 'Fp2', 'F3', 'F4', 'F7', 'F8', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2']
pairs = [('Fp1', 'Fp2'), ('F3', 'F4'), ('F7', 'F8'), ('C3', 'C4'), ('P3', 'P4'), ('O1', 'O2')]
idces = ['Fp1', 'Fp2', 'F3', 'F4', 'F7', 'F8', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'Fp1/Fp2', 'F3/F4', 'F7/F8', 'C3/C4', 'P3/P4', 'O1/O2']
subjects = []
idces1 = list()
for channel in channels:
    for band in band_dict.keys():
        idces1.append(channel + "a " + band)
for channel in channels:
    for band in band_dict.keys():
        idces1.append(channel + "r " + band)
for channel in idces[12:]:
    for band in band_dict.keys():
        idces1.append(channel + " " + band)
df1ec = pd.DataFrame(index=subjects, columns=idces1)
df1eo = pd.DataFrame(index=subjects, columns=idces1)
display(df1ec)

,Fp1a delta,Fp1a theta,Fp1a alpha,Fp1a beta,Fp1a gamma,Fp2a delta,Fp2a theta,Fp2a alpha,Fp2a beta,Fp2a gamma,...,P3/P4 delta,P3/P4 theta,P3/P4 alpha,P3/P4 beta,P3/P4 gamma,O1/O2 delta,O1/O2 theta,O1/O2 alpha,O1/O2 beta,O1/O2 gamma


In [3]:
for i in range(322):
    for j in ["_EC", "_EO"]:
        fileurlfdt = "https://ftp.gwdg.de/pub/misc/MPI-Leipzig_Mind-Brain-Body-LEMON/EEG_MPILMBB_LEMON/EEG_Preprocessed_BIDS_ID/EEG_Preprocessed/sub-0" + str(10000 + i) + j + ".fdt"
        fileurlset = "https://ftp.gwdg.de/pub/misc/MPI-Leipzig_Mind-Brain-Body-LEMON/EEG_MPILMBB_LEMON/EEG_Preprocessed_BIDS_ID/EEG_Preprocessed/sub-0" + str(10000 + i) + j + ".set"
        try:
            wget.download(fileurlfdt, 'sub-0' + str(10000 + i) + j + '.fdt')
            wget.download(fileurlset, 'sub-0' + str(10000 + i) + j + '.set')
            raw = mne.io.read_raw_eeglab('sub-0' + str(10000 + i) + j + '.set')
            subject = "0" + str(10000 + i)
            df_r = pd.DataFrame(index=idces, columns=band_dict.keys())
            df_a = pd.DataFrame(index=channels, columns=band_dict.keys())
            for channel in channels:
                spectra, freqs = psd_welch(raw, fmin=1.5, fmax=45, n_fft=int(raw.info['sfreq']*4), picks=channel, verbose=False)
                for band, lims in band_dict.items():
                    low, high = lims
                    idx_band = np.logical_and(freqs > low, freqs <= high)
                    freq_res = freqs[1] - freqs[0]
                    total_power = simps(spectra[0], dx=freq_res)
                    abs_power = simps(spectra[0][idx_band], dx=freq_res)
                    df_a[band][channel] = abs_power * 10**12
                    rel_power = abs_power / total_power
                    df_r[band][channel] = rel_power
            for pair in pairs:
                left, right = pair
                for band in band_dict.keys():
                    df_r[band][left + "/" + right] = df_a[band][left] / (df_a[band][left] + df_a[band][right])
            subjects.append(subject)
            df = pd.DataFrame(index=[subject], columns=idces1)
            for band in band_dict.keys():
                for channel in channels:
                    df[channel + "a " + band][subject] = df_a[band][channel]
                    df[channel + "r " + band][subject] = df_r[band][channel]
                for channel in idces[12:]:
                        df[channel + " " + band][subject] = df_r[band][channel]
            os.remove('sub-0' + str(10000 + i) + j + '.fdt')
            os.remove('sub-0' + str(10000 + i) + j + '.set')
            if j == "_EC":
                df1ec = df1ec.append(df)
            elif j == "_EO":
                df1eo = df1eo.append(df)
        except:
            pass

100% [..........................................................................] 1198784 / 1198784Reading sub-010002_EC.fdt


<ipython-input-3-eddedfb9c70e>:8: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab('sub-0' + str(10000 + i) + j + '.set')
<ipython-input-3-eddedfb9c70e>:8: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab('sub-0' + str(10000 + i) + j + '.set')


100% [..........................................................................] 1186864 / 1186864Reading sub-010002_EO.fdt


<ipython-input-3-eddedfb9c70e>:8: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab('sub-0' + str(10000 + i) + j + '.set')
<ipython-input-3-eddedfb9c70e>:8: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab('sub-0' + str(10000 + i) + j + '.set')


100% [..........................................................................] 1201152 / 1201152Reading sub-010003_EC.fdt


...

<ipython-input-3-eddedfb9c70e>:8: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab('sub-0' + str(10000 + i) + j + '.set')
<ipython-input-3-eddedfb9c70e>:8: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab('sub-0' + str(10000 + i) + j + '.set')


100% [..........................................................................] 1207456 / 1207456Reading sub-010321_EC.fdt


<ipython-input-3-eddedfb9c70e>:8: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab('sub-0' + str(10000 + i) + j + '.set')
<ipython-input-3-eddedfb9c70e>:8: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab('sub-0' + str(10000 + i) + j + '.set')


100% [..........................................................................] 1195160 / 1195160Reading sub-010321_EO.fdt


<ipython-input-3-eddedfb9c70e>:8: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab('sub-0' + str(10000 + i) + j + '.set')
<ipython-input-3-eddedfb9c70e>:8: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab('sub-0' + str(10000 + i) + j + '.set')


In [4]:
df1ec

,Fp1a delta,Fp1a theta,Fp1a alpha,Fp1a beta,Fp1a gamma,Fp2a delta,Fp2a theta,Fp2a alpha,Fp2a beta,Fp2a gamma,...,P3/P4 delta,P3/P4 theta,P3/P4 alpha,P3/P4 beta,P3/P4 gamma,O1/O2 delta,O1/O2 theta,O1/O2 alpha,O1/O2 beta,O1/O2 gamma
010002,2.47537,1.63864,3.01564,11.5988,3.48337,2.28259,1.55222,2.66411,7.49358,1.58842,...,0.478741,0.500657,0.550677,0.566536,0.580211,0.495397,0.495227,0.520955,0.557818,0.472947
010003,19.9222,21.1276,39.0193,6.42745,0.734595,21.0734,22.0507,37.8635,6.99701,0.901528,...,0.467111,0.462132,0.490572,0.481299,0.49005,0.480509,0.4949,0.492222,0.510392,0.61586
010004,7.44159,7.14334,11.5542,6.02234,0.700233,7.538,6.6857,9.01654,6.24996,0.932831,...,0.523174,0.535934,0.554881,0.513354,0.50912,0.502332,0.495862,0.46425,0.479605,0.473999
010005,9.80876,6.9681,11.1188,10.1388,0.35858,9.05805,6.6107,10.779,9.74515,0.350154,...,0.465592,0.500293,0.470638,0.482173,0.483798,0.516039,0.523242,0.544121,0.498107,0.504212
010006,3.14293,4.25457,8.85225,7.04499,0.569409,3.26639,4.24416,8.76698,7.11489,0.524147,...,0.515464,0.507248,0.411856,0.471365,0.498517,0.514724,0.508148,0.471631,0.503539,0.503567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
010316,4.77519,3.58725,3.62605,1.9754,0.432544,5.34542,3.56078,4.09701,4.05123,1.23694,...,0.470399,0.479859,0.370586,0.459771,0.481945,0.513476,0.508598,0.381715,0.44799,0.487156
010317,1.74882,1.18847,2.27361,1.12804,0.296815,2.04289,1.33041,2.42732,1.01091,0.228158,...,0.448729,0.4446,0.402036,0.434526,0.475279,0.512463,0.512346,0.489442,0.532473,0.602015
010318,7.09551,3.90635,4.30373,6.29743,0.758554,8.31703,4.40364,4.70112,7.39119,0.949698,...,0.453108,0.409012,0.34304,0.410707,0.478306,0.511121,0.488979,0.555404,0.515656,0.510222
010319,10.9183,5.05063,23.6351,10.075,0.655482,11.343,5.36051,27.1462,11.4445,0.700679,...,0.480541,0.488522,0.421483,0.440575,0.439214,0.517001,0.51532,0.514562,0.506926,0.504901


In [5]:
df1ec.to_csv('EEG_MPIL_dataset_powers_EC.csv', index=True)

In [6]:
df1eo

,Fp1a delta,Fp1a theta,Fp1a alpha,Fp1a beta,Fp1a gamma,Fp2a delta,Fp2a theta,Fp2a alpha,Fp2a beta,Fp2a gamma,...,P3/P4 delta,P3/P4 theta,P3/P4 alpha,P3/P4 beta,P3/P4 gamma,O1/O2 delta,O1/O2 theta,O1/O2 alpha,O1/O2 beta,O1/O2 gamma
010002,2.96329,1.75886,2.74188,14.8829,4.59773,3.04579,1.77287,2.37401,7.50817,1.59238,...,0.473699,0.487901,0.505614,0.567835,0.557093,0.465475,0.468386,0.537042,0.543826,0.463206
010003,6.51389,5.82957,14.3012,4.51771,0.849736,6.65501,5.80828,13.9324,3.75329,0.50087,...,0.485737,0.458422,0.443813,0.466147,0.500808,0.509675,0.498215,0.478986,0.532515,0.634469
010004,4.534,4.10381,8.20201,10.9341,3.46816,2.5084,2.68571,4.97799,7.41909,1.73779,...,0.536361,0.531348,0.536021,0.507245,0.510665,0.506654,0.492829,0.484946,0.462731,0.474542
010005,13.001,6.37637,8.47907,10.9351,1.12015,10.5287,5.09228,7.25741,9.55564,1.14813,...,0.458471,0.484618,0.518575,0.480005,0.473664,0.502523,0.514538,0.537265,0.500549,0.485857
010006,2.75109,2.15725,4.77844,7.53114,2.04347,2.99856,2.41007,5.31202,11.6799,4.59419,...,0.528464,0.519394,0.478897,0.516,0.495676,0.523968,0.515604,0.505154,0.516178,0.496556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
010315,3.50864,2.24501,2.86695,3.80737,0.41868,4.26623,2.46908,3.21102,4.36911,0.746186,...,0.469807,0.493699,0.523106,0.478229,0.527235,0.465137,0.481502,0.487168,0.478506,0.496092
010316,5.66838,4.37441,2.16633,1.63037,0.352707,6.15741,4.67368,2.31218,1.82235,0.418161,...,0.496886,0.504561,0.494965,0.491063,0.470268,0.50139,0.504902,0.496936,0.495394,0.482235
010318,4.3709,2.28835,1.69924,5.64724,0.892734,6.2487,2.95255,2.14379,7.06724,1.24966,...,0.461284,0.459103,0.472405,0.45849,0.47013,0.497453,0.485832,0.499386,0.501177,0.508355
010319,12.2634,4.8614,13.1688,14.8624,1.86962,11.4096,4.53821,13.7349,10.7702,0.730452,...,0.454806,0.483272,0.509207,0.443675,0.472377,0.49469,0.491955,0.558415,0.510408,0.507811


In [7]:
df1eo.to_csv('EEG_MPIL_dataset_powers_EO.csv', index=True)

In [8]:
wget.download('https://ftp.gwdg.de/pub/misc/MPI-Leipzig_Mind-Brain-Body-LEMON/Behavioural_Data_MPILMBB_LEMON/META_File_IDs_Age_Gender_Education_Drug_Smoke_SKID_LEMON.csv', 'MPIL_subject_data.csv')

100% [..............................................................................] 39364 / 39364

'MPIL_subject_data (1).csv'

In [9]:
df2 = pd.read_csv('MPIL_subject_data.csv', index_col='ID')
df2

,Gender_ 1=female_2=male,Age,Handedness,Education,DRUG,DRUG_0=negative_1=Positive,Unnamed: 7,Smoking,"Smoking_num_(Non-smoker=1, Occasional Smoker=2, Smoker=3)",SKID_Diagnoses,SKID_Diagnoses 1,SKID_Diagnoses 2,Comments_SKID_assessment,Hamilton_Scale,BSL23_sumscore,BSL23_behavior,AUDIT,Standard_Alcoholunits_Last_28days,Alcohol_Dependence_In_1st-3rd_Degree_relative,Relationship_Status
ID,,,,,,,,,,,,,,,,,,,,
sub-010017,2,20-25,right,Gymnasium,negative,0.0,NaN,non-smoker,1.0,none,#CODE:V71.09 #DESC:No Diagnosis or Condition ...,NaN,NaN,0.0,NaN,NaN,12.0,47,No,No
sub-010015,2,20-25,right,Gymnasium,negative,0.0,NaN,occasional smoker,2.0,none,#CODE:V71.09 #DESC:No Diagnosis or Condition ...,NaN,NaN,0.0,NaN,NaN,12.0,45,Yes,Yes
sub-010100,2,25-30,right,Gymnasium,negative,0.0,NaN,occasional smoker,2.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,0.0,12,unknown,Yes
sub-010003,1,20-25,right,Gymnasium,negative,0.0,NaN,non-smoker,1.0,none,#CODE:V71.09 #DESC:No Diagnosis or Condition ...,NaN,NaN,3.0,NaN,NaN,1.0,0,No,No
sub-010022,2,20-25,right,Gymnasium,negative,0.0,NaN,non-smoker,1.0,none,#CODE:V71.09 #DESC:No Diagnosis or Condition ...,NaN,NaN,2.0,NaN,NaN,7.0,22,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sub-010005,2,25-30,right,Gymnasium,negative,0.0,NaN,non-smoker,1.0,none,#CODE:V71.09 #DESC:No Diagnosis or Condition ...,NaN,NaN,0.0,4.0,0.0,1.0,1,No,No
sub-010231,2,20-25,left,Gymnasium,negative,0.0,NaN,non-smoker,1.0,none,#CODE:V71.09 #DESC:No Diagnosis or Condition ...,NaN,"suspected diagnosis: panic disorder, alcohol a...",1.0,4.0,1.0,5.0,18,no,no
sub-010232,2,25-30,left,Gymnasium,negative,0.0,NaN,non-smoker,1.0,none,#CODE:V71.09 #DESC:No Diagnosis or Condition ...,NaN,NaN,5.0,2.0,1.0,3.0,29,no,yes
